In [ ]:
import gc
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as pyplot
import numpy as np
import pandas as pd
import scipy as sp
import scipy.constants
from matplotlib import cm, ticker
from numpy import ma

In [ ]:
%matplotlib inline

pyplot.style.use(['dark_background'])
mpl.rcParams["axes.grid"] = True
mpl.rcParams["grid.linewidth"] = 0.5
mpl.rcParams["grid.alpha"] = 0.5
mpl.rcParams["grid.color"] = "FFFFFF"
mpl.rcParams["figure.figsize"] = [10.0, 10.0 / sp.constants.golden]

In [ ]:
OUTPUT_DIR = Path("/tmp/leptogenesis_sp")

# Number Density Solutions

In [ ]:
data_n = pd.read_csv(OUTPUT_DIR / "n.csv")
XSCALE = "log"

gc.collect()

In [ ]:
fig, ax = pyplot.subplots()
# ax.set_xscale('log')
ax.set_yscale("log")
data_n.plot(x="step", y="beta", ax=ax)

pyplot.show()

In [ ]:
## B-L
fig, ax = pyplot.subplots()
ax.set_xscale(XSCALE)
ax.set_yscale("log")
data_n.abs().plot(x="beta", y=data_n.columns[2:3], ax=ax)
ax.set_ylim(bottom=1e-20)

## Number density
fig, ax = pyplot.subplots()
ax.set_xscale(XSCALE)
ax.set_yscale("log")
data_n.plot(x="beta", y=data_n.columns[2::3], ax=ax)
ax.set_ylim(bottom=1e-10, top=1e10)

## Equilibrium Number Density
fig, ax = pyplot.subplots()
ax.set_xscale(XSCALE)
ax.set_yscale("linear")
data_n.plot(x="beta", y=data_n.columns[3::3], ax=ax)

## Change in Number Density
fig, ax = pyplot.subplots()
ax.set_xscale(XSCALE)
ax.set_yscale("log")
data_n.abs().plot(x="beta", y=data_n.columns[4::3], ax=ax)
ax.set_ylim(bottom=1e-20)

pyplot.show()

In [ ]:
df = data_n.iloc[:,[0,1,10]]
df[df.iloc[:,2].abs() > 1e-3]

# Interactions

## 1-to-2 Processes

In [ ]:
data = pd.read_csv(OUTPUT_DIR / "decay.csv")

gc.collect()

In [ ]:
fig, ax = pyplot.subplots()

ax.set_xscale("log")
ax.set_yscale("log")

data.abs().plot(x="beta", y=data.columns[2:], ax=ax)

ax.set_ylim(bottom=1e-20)
ax.set_xlim(left=1e-13, right=1e-7)

## 2-to-2 Processes

In [ ]:
data = pd.read_csv(OUTPUT_DIR / "scattering_NLQq.csv")

gc.collect()

In [ ]:
fig, ax = pyplot.subplots()

ax.set_xscale("log")
ax.set_yscale("log")

data.abs().plot(x="beta", y=data.columns[2:], ax=ax)

ax.set_ylim(bottom=1e-10, top=1e20)
ax.set_xlim(left=1e-13, right=1e-7)

# Scan

In [ ]:
data = pd.read_csv(OUTPUT_DIR / "scan.csv")
data.sort_values(by=["y", "m"], inplace=True)

gc.collect()

In [ ]:
fig, ax = pyplot.subplots()

x = data["y"].unique()
y = data["m"].unique()
z = data["B-L"].values.reshape(x.size, y.size)
z = ma.masked_where(z <= 0, z)

ax.set_xlabel("$y_v$")
ax.set_xscale("log")
ax.set_ylabel("$N_1$ mass [GeV]")
ax.set_yscale("log")
cs = ax.contourf(x, y, z, locator=ticker.LogLocator())
cbar = fig.colorbar(cs)
cbar.set_label("B-L")

pyplot.show()